In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential,load_model
from keras import layers,Input
from keras.losses import binary_crossentropy
from keras.layers import Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import keras.backend as K

## Old data

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [93]:
state0=np.load('../Data/ML/121623/Wstate0_1µs.npy')
state1=np.load('../Data/ML/121623/Wstate1_1µs.npy')
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state1.shape)

(5000, 2)


In [94]:
x=np.concatenate((state0,state1),axis=0)
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

(10000, 2)
(10000,)


In [95]:
print(x[:,0].max(),x[:,0].min())
print(x[:,1].max(),x[:,1].min())
'''Let min and max be 2^28'''

x=(x+(2**28))/(2**29)
print(x[:,0].max(),x[:,0].min())
print(x[:,1].max(),x[:,1].min())
# 202 875 740

202875740.38027424 -210086248.8832826
141376509.25470012 -176239963.77299935
0.8778855137159568 0.1086838675974261
0.7633342691784727 0.17172748637758317


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)
sc = StandardScaler()
sc = StandardScaler()
x=x[:,700:2700,:].sum(axis=1)
X= sc.fit_transform(x)
# X_test = sc.transform(x_test)
pca = PCA(n_components = 1)
X = pca.fit_transform(X)
print (pca.explained_variance_)
print (pca.explained_variance_ratio_)
print (pca.explained_variance_ratio_.cumsum())
# X_test = pca.transform(X_test)
print("X shape: {}".format(X.shape))
x_red =X[y[:] == 1, 0]  # Use y[:, 0] for indexing
x_blue = X[y[:] == 0, 0] 

# Create a histogram with two different colors
plt.hist(x_blue, bins=50, color='blue', alpha=0.7, label='y=0')
plt.hist(x_red, bins=50, color='red', alpha=0.7, label='y=1')
# Add labels and legend
plt.xlabel('Scaled values')
plt.ylabel('Frequency')
plt.legend()

# Show the plot
plt.show()

In [96]:

# st,ed=700,4700
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())

# x_train=np.mean(x_train,axis=1)
# x_test=np.mean(x_test,axis=1)

# x_train=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_test=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
print(x_train.shape)

(7000, 2)
3515.0
(7000, 2)


## Logistic Regression

In [97]:
# x_trainC=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_testC=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)

0.844


## SVM

In [98]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))
pred=model.predict(x_test)

0.845


## GMM

In [ ]:
Y=list(zip(np.append(np.mean(state0[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,0],axis=1)),np.append(np.mean(state0[:,st:ed,1],axis=1),np.mean(state1[:,st:ed,1],axis=1))))
gmm_Y=GaussianMixture(n_components=2,covariance_type='full').fit(Y)
bitstring=np.split(gmm_Y.predict(Y),2)
print('P(0|0)=',len(bitstring[0][bitstring[0]==0])/len(bitstring[0]))
print('P(1|1)=',len(bitstring[1][bitstring[1]==1])/len(bitstring[1]))
X=list(zip(np.mean(state0[:,st:ed,0],axis=1),np.mean(state0[:,st:ed,1],axis=1)))
gmm_Y_1=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)
X=list(zip(np.mean(state1[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,1],axis=1)))
gmm_Y_2=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)

## NN

In [105]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2), name='Input Layer'))
    model.add(Dense(8,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model
arch().summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 HiddenLayer1 (Dense)        (None, 8)                 24        
                                                                 
 HiddenLayer2 (Dense)        (None, 4)                 36        
                                                                 
 OuputLayer (Dense)          (None, 1)                 5         
                                                                 
Total params: 65 (260.00 Byte)
Trainable params: 65 (260.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [106]:
model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/tmp/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [107]:
H=model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          # validation_split=0.2,
          epochs=100,batch_size=64,
        #   callbacks=callbacks_list,
          verbose=1)

Epoch 1/100
110/110 [==============================] - 1s 3ms/step - loss: 0.6816 - accuracy: 0.5021 - val_loss: 0.6758 - val_accuracy: 0.4957
Epoch 2/100
110/110 [==============================] - 0s 2ms/step - loss: 0.6670 - accuracy: 0.5440 - val_loss: 0.6567 - val_accuracy: 0.6270
Epoch 3/100
110/110 [==============================] - 0s 2ms/step - loss: 0.6430 - accuracy: 0.6924 - val_loss: 0.6275 - val_accuracy: 0.6863
Epoch 4/100
110/110 [==============================] - 0s 2ms/step - loss: 0.6102 - accuracy: 0.7511 - val_loss: 0.5923 - val_accuracy: 0.7477
Epoch 5/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5748 - accuracy: 0.7747 - val_loss: 0.5571 - val_accuracy: 0.7920
Epoch 6/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 0.8044 - val_loss: 0.5261 - val_accuracy: 0.8027
Epoch 7/100
110/110 [==============================] - 0s 2ms/step - loss: 0.5122 - accuracy: 0.8147 - val_loss: 0.5000 - val_accuracy: 0.8313

In [ ]:
N = 100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="test_loss")
plt.title("Singlechannel")
plt.xlabel("Epoch #")
plt.ylabel("loss")
plt.legend(loc="lower left")

N = 100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="test_accuracy")
plt.title("Singlechannel")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")

In [ ]:
model=load_model('../Model/NN2/tmp/29-0.7716.h5', compile=False)
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

In [ ]:
np.save('../x_text.npy',x_test)
np.save('../y_test.npy',y_test)

## GDA

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminantAnalysis:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.mean(y == c) for c in self.classes])

        self.means = []
        self.cov_matrices = []
        for c in self.classes:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            cov_matrix_c = np.cov(X_c, rowvar=False)
            self.means.append(mean_c)
            self.cov_matrices.append(cov_matrix_c)

    def predict(self, X):
        posteriors = []

        for c in self.classes:
            class_prior = self.class_priors[int(c)]
            mean = self.means[int(c)]
            cov_matrix = self.cov_matrices[int(c)]
            mvn = multivariate_normal(mean=mean, cov=cov_matrix)
            posterior = class_prior * mvn.pdf(X)
            posteriors.append(posterior)

        posteriors = np.array(posteriors).T
        predicted_labels = np.argmax(posteriors, axis=1)

        return predicted_labels



# Initialize and train GDA
gda = GaussianDiscriminantAnalysis()
gda.fit(x_train, y_train)

# Make predictions
y_pred = gda.predict(x_train)
# Calculate accuracy
accuracy = np.mean(y_pred == y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")

## MultiChannel

In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
# def plotCluster(s0I,s0Q,s1I,s1Q,e0I,e0Q,e1I,e1Q,f0I,f0Q,f1I,f1Q,):
#     fig = make_subplots(rows=1, cols=1)
#     trace0 = go.Scatter(x=s0I, y=s0Q, name='0-500_|0>',mode='markers',line=dict(color='blue'),marker=dict(size=8, symbol='circle-open'))
#     trace1 = go.Scatter(x=s1I, y=s1Q, name='0-500_|1>',mode='markers',line=dict(color='red'),marker=dict(size=8,symbol='square-open'))
#     trace2 = go.Scatter(x=e0I, y=e0Q, name='500-1000_|0>',mode='markers',line=dict(color='black'),marker=dict(size=8,symbol='star-open'))
#     trace3 = go.Scatter(x=e1I, y=e1Q, name='500-1000_|1>',mode='markers',line=dict(color='green'),marker=dict(size=8, symbol='diamond-open'))
#     trace4 = go.Scatter(x=f0I, y=f0Q, name='1000-1500_|0>',mode='markers',line=dict(color='orange'),marker=dict(size=8,symbol='square-cross-open'))
#     trace5 = go.Scatter(x=f1I, y=f1Q, name='1000-1500_|1>',mode='markers',line=dict(color='pink'),marker=dict(size=8, symbol='diamond-cross-open'))
#     fig.add_trace(trace0)
#     fig.add_trace(trace1)
#     fig.add_trace(trace2)
#     fig.add_trace(trace3)
#     fig.add_trace(trace4)
#     fig.add_trace(trace5)
#     # Set the layout
#     fig.update_layout(
#         title="Signals 0-500-1000-1500",
#         xaxis_title="I",
#         yaxis_title="Q",
#         showlegend=True,
#         width=800,  # Set the width of the plot
#         height=400,  # Set the height of the plot
#     )
#     return fig  

def plotCluster(s0I,s0Q,s1I,s1Q, name1, name2):
    fig = make_subplots(rows=1, cols=1)
    trace0 = go.Scatter(x=s0I, y=s0Q, name=name1,mode='markers',line=dict(color='blue'),marker=dict(size=8, symbol='circle-open'))
    trace1 = go.Scatter(x=s1I, y=s1Q, name=name2,mode='markers',line=dict(color='red'),marker=dict(size=8,symbol='square-open'))
    fig.add_trace(trace0)
    fig.add_trace(trace1)
    # Set the layout
    fig.update_layout(
        title="Signals",
        xaxis_title="I",
        yaxis_title="Q",
        showlegend=True,
        width=800,  # Set the width of the plot
        height=400,  # Set the height of the plot
    )
    return fig  

In [ ]:
# state0_500= np.mean(state0[:,700:1700,:],axis=1)
# state1_500= np.mean(state1[:,700:1700,:],axis=1)
# state0_1000= np.mean(state0[:,1700:2700,:],axis=1)
# state1_1000= np.mean(state1[:,1700:2700,:],axis=1)
# state0_1500= np.mean(state0[:,2700:3700,:],axis=1)
# state1_1500= np.mean(state1[:,2700:3700,:],axis=1)
# state0_4=np.hstack((state0_500,state0_1000,state0_1500))
# state1_4=np.hstack((state1_500,state1_1000,state1_1500))

# indices = np.arange(6000)
# np.random.shuffle(indices)
# selected_indices = indices[:1000]
# random0 = state0_4[selected_indices]
# random1 = state1_4[selected_indices]


# print(random0.shape)

In [ ]:
state0_100= np.sum(state0[:,1500:1700,:],axis=1)
state1_100= np.sum(state1[:,1500:1700,:],axis=1)
state0_200= np.sum(state0[:,1900:2100,:],axis=1)
state1_200= np.sum(state1[:,1900:2100,:],axis=1)
state0_300= np.sum(state0[:,2100:2300,:],axis=1)
state1_300= np.sum(state1[:,2100:2300,:],axis=1)
state0_400= np.sum(state0[:,2300:2500,:],axis=1)
state1_400= np.sum(state1[:,2300:2500,:],axis=1)
state0_500= np.sum(state0[:,2500:2700,:],axis=1)
state1_500= np.sum(state1[:,2500:2700,:],axis=1)
state0_4=np.hstack((state0_100,state0_200,state0_300,state0_400,state0_500))
state1_4=np.hstack((state1_100,state1_200,state1_300,state1_400,state1_500))

indices = np.arange(6000)
np.random.shuffle(indices)
selected_indices = indices[:1000]
random0 = state0_4[selected_indices]
random1 = state1_4[selected_indices]


print(random0.shape)

In [ ]:
# plotCluster(random0[:,0], random0[:,1], random1[:,0], random1[:,1], random0[:, 2], random0[:, 3], random1[:,2], random1[:,3],random0[:, 4], random0[:, 5], random1[:,4], random1[:,5])
plotCluster(random0[:,0], random0[:,1], random1[:,0], random1[:,1],'500-600_|0>', '500-600_|1>').show()
plotCluster(random0[:, 2], random0[:, 3], random1[:,2], random1[:,3],'600-700_|0>', '600-700_|1>').show()
plotCluster(random0[:, 4], random0[:, 5], random1[:,4], random1[:,5],'700-800_|0>', '700-800_|1>').show()
plotCluster(random0[:,6], random0[:,7], random1[:,6], random1[:,7],'800-900_|0>', '800-900_|1>').show()
plotCluster(random0[:, 8], random0[:, 9], random1[:,8], random1[:,9],'900-1000_|0>', '900-1000_|1>').show()
# plotCluster(np.sum(state0[selected_indices,1700:3700, 0],axis=1), np.sum(state0[selected_indices,1700:3700, 1],axis=1), np.sum(state1[selected_indices,1700:3700,0],axis=1), np.sum(state1[selected_indices,1700:3700,1],axis=1),'900-1000_|0>', '900-1000_|1>').show()


In [ ]:
x=np.vstack((state0_4,state1_4))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())

In [ ]:
def arch():
    model=Sequential()
    model.add(Input(shape=(10), name='Input Layer'))
    model.add(Dense(16,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model
arch().summary()

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/tmp/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

H=model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=100,batch_size=64,
        #   callbacks=callbacks_list,
          verbose=1)
# model=load_model('../Model/NN2/tmp/29-0.7716.h5', compile=False)
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i]<=0.5:
            count1+=1
print(count,count1)

count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

In [ ]:
N = 100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="test_loss")
plt.title("Multichannel")
plt.xlabel("Epoch #")
plt.ylabel("loss")
plt.legend(loc="lower left")

N = 100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="test_accuracy")
plt.title("Multichannel")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


threshold = 0.5
y_pred = (y_pred > threshold).astype(int)
conf_mat = confusion_matrix(y_test, y_pred)

# Calculate percentages
conf_mat_percentage = conf_mat / conf_mat.sum(axis=1)[:, np.newaxis]

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_mat)

# Create a heatmap for better visualization with percentages
plt.figure(figsize=(6, 4))
sns.heatmap(conf_mat_percentage, annot=True, fmt=".2%", cmap="Blues", xticklabels=["Predicted 0", "Predicted 1"], yticklabels=["Actual 0", "Actual 1"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Percentages)")
plt.show()

## New

In [ ]:
state0=np.stack((np.load('../Data/neel_readout/120923/state0_15µs_cos.npy').real,np.load('../Data/neel_readout/120923/state0_15µs_cos.npy').imag),axis=1)
state1=np.stack((np.load('../Data/neel_readout/120923/state1_15µs_cos.npy').real,np.load('../Data/neel_readout/120923/state1_15µs_cos.npy').imag),axis=1)
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)

In [ ]:
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

min,max= -(2**20),(2**20)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))
x= (x-min)/(max-min)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.80, random_state=45)
print(x_train.shape)
print(y_train.sum())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X= sc.fit_transform(x)
pca = PCA(n_components = 1)
X = pca.fit_transform(X)
print (pca.explained_variance_)
print (pca.explained_variance_ratio_)
print (pca.explained_variance_ratio_.cumsum())
# X_test = pca.transform(X_test)
print("X shape: {}".format(X.shape))
x_red =X[y[:] == 1, 0]  # Use y[:, 0] for indexing
x_blue = X[y[:] == 0, 0] 

# Create a histogram with two different colors
plt.hist(x_blue, bins=50, color='blue', alpha=0.7, label='y=0')
plt.hist(x_red, bins=50, color='red', alpha=0.7, label='y=1')
# Add labels and legend
plt.xlabel('Scaled values')
plt.ylabel('Frequency')
plt.legend()

# Show the plot
plt.show()

In [ ]:
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)

In [ ]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))
pred=model.predict(x_test)

In [ ]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2), name='Input Layer'))
    model.add(Dense(8,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model
arch().summary()

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/tmp/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

H=model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=100,batch_size=64,
        #   callbacks=callbacks_list,
          verbose=1)
# model=load_model('../Model/NN2/tmp/29-0.7716.h5', compile=False)
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i]<=0.5:
            count1+=1
print(count,count1)

count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)